In [2]:
pip install asyncpraw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
import aiohttp
import asyncpraw
import pandas as pd
from datetime import datetime, timezone
import asyncio
import csv

async def get_auth_credentials():
    credentials = {
        "client_id": input("Enter Reddit client ID: "),
        "client_secret": input("Enter Reddit client secret: "),
        "user_agent": input("Enter Reddit user agent: ")
    }
    return credentials

async def reddit_authenticate(credentials, session):
    try:
        reddit = asyncpraw.Reddit(
            client_id=credentials['client_id'],
            client_secret=credentials['client_secret'],
            user_agent=credentials['user_agent'],
            requestor_kwargs={'session': session}
        )
        await reddit.user.me()
        print("Authentication successful.")
        return reddit
    except Exception as e:
        print(f"Authentication failed: {e}")
        exit(1)

async def fetch_subreddit_posts(reddit, subreddit_name, limit=2000):
    posts_data = []
    try:
        subreddit = await reddit.subreddit(subreddit_name)
        async for submission in subreddit.new(limit=limit):
            post_content = submission.selftext if submission.is_self else ""
            posts_data.append({
                'author': str(submission.author),
                'title': submission.title,
                'comments': submission.num_comments,
                'created_at': datetime.fromtimestamp(submission.created_utc, tz=timezone.utc),
                'content': post_content,
                'url': submission.url
            })
    except Exception as e:
        print(f"Error fetching posts for subreddit {subreddit_name}: {e}")
    return posts_data

async def display_posts(posts):
    if posts:
        df = pd.DataFrame(posts)
        print(df.head(10))
    else:
        print("No posts found.")

async def save_posts_to_csv(posts):
    if not posts:
        print("No posts to save.")
        return
    filename = input("Enter filename (with .csv extension): ")
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=posts[0].keys())
        writer.writeheader()
        writer.writerows(posts)
    print(f"Posts saved to {filename}")

async def main():
    credentials = await get_auth_credentials()

    async with aiohttp.ClientSession() as session:
        reddit = await reddit_authenticate(credentials, session)
        subreddit_name = input("Enter the target subreddit name: ")
        posts = await fetch_subreddit_posts(reddit, subreddit_name)
        await display_posts(posts)
        await save_posts_to_csv(posts)

if __name__ == "__main__":
    try:
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.run(main())
    except Exception as e:
        print(f"An error occurred: {e}")

